In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

For this project i have been working in kaggle ,it ideal to run the notebook in kaggle but it's ur wish, if you face any error in running the notebook you use my kaggle notebook ,the link for th kaggle notebook is attache below
https://www.kaggle.com/code/sahiltarlana2601/numeral-ipynb

### Importing important Libraries

In [12]:
import pandas as pd
import re
import torch
import torch.nn as nn
import spacy
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader

### Constants Explanation:

1. **`FIXED_NUMERAL_LENGTH == 8`**:
   - This sets a fixed length of 8 characters for numerals. If a numeral is shorter, it gets padded with zeros; if it's longer, it's cut off. This ensures all numerals have the same length when processed.

2. **`MAX_ASPECT_DISTANCE == 10`**:
   - This limits the maximum distance between a numeral and the subject (aspect) of a sentence to 10. If the distance is greater, it gets capped at 10 to keep the values manageable for the model.

These constants help standardize inputs and control values during numeral processing.

In [13]:
# Constants
FIXED_NUMERAL_LENGTH = 8  # Adjust 'o' as needed
MAX_ASPECT_DISTANCE = 10  # Maximum distance for AspDis embedding

In [28]:
nlp = spacy.load("en_core_web_sm")

loading the data to process

In [14]:
data= pd.read_csv("/kaggle/input/apollo-news/apollo_aspect.csv")

In [15]:
data.head()

,date,news,Predicted Sentiment,Sentiment Probability,aspect_labelling
0,2018-02-01,"In an interview with CNBC-TV18, Shobana Kamine...",neutral,0.577611,"[[('Apollo Hospitals', 'companies', 'neutral',..."
1,2018-02-15,Apollo Hospitals Enterprise today reported a 7...,negative,0.973464,"[[('expenses', 'financial_performance', 'neutr..."
2,2018-06-06,Hold Apollo Hospitals Enterprise Ltd. at a p...,neutral,0.939086,"[[('Apollo Hospitals', 'companies', 'neutral',..."
3,2018-07-06,Apollo Hospitals Enterprise Ltd has acquired a...,neutral,0.903377,"[[('stake', 'stock_market', 'neutral', 0.91562..."
4,2018-09-18,"Apollo Hospitals Chennai, a flagship of Apollo...",neutral,0.828442,"[[('Apollo Hospitals', 'companies', 'neutral',..."


In [16]:
print (data.columns)

Index(['date', 'news', 'Predicted Sentiment', 'Sentiment Probability',
       'aspect_labelling'],
      dtype='object')


In [17]:
print(data['news'])

0      In an interview with CNBC-TV18, Shobana Kamine...
1      Apollo Hospitals Enterprise today reported a 7...
2      Hold  Apollo Hospitals Enterprise Ltd.  at a p...
3      Apollo Hospitals Enterprise Ltd has acquired a...
4      Apollo Hospitals Chennai, a flagship of Apollo...
                             ...                        
468    The report is based on the agency’s sample of ...
469    Apollo Hospitals will add 500-600 offline phar...
470    Apollo’s solid organ transplant program includ...
471    Motilal Oswal is bullish on Apollo Hospitals r...
472    Apollo Hospital Enterprise’s stock (₹4,865.5) ...
Name: news, Length: 473, dtype: object


In [18]:
ndata=data[['news']]
ndata

,news
0,"In an interview with CNBC-TV18, Shobana Kamine..."
1,Apollo Hospitals Enterprise today reported a 7...
2,Hold Apollo Hospitals Enterprise Ltd. at a p...
3,Apollo Hospitals Enterprise Ltd has acquired a...
4,"Apollo Hospitals Chennai, a flagship of Apollo..."
...,...
468,The report is based on the agency’s sample of ...
469,Apollo Hospitals will add 500-600 offline phar...
470,Apollo’s solid organ transplant program includ...
471,Motilal Oswal is bullish on Apollo Hospitals r...


In [19]:
ndata['news'][4]

'Apollo Hospitals Chennai, a flagship of Apollo Hospitals Enterprise Ltd and the country’s first corporate hospital, marked its 35th anniversary here on Tuesday.Launched in 1983, it now has 904 beds in'

### `extract_numerals` Function:

This function extracts numerals (numbers) from a given sentence using regular expressions.

- **Regex pattern** `r"[-]?\d*\.?\d+"`:
  - `[-]?`: Optional minus sign for negative numbers.
  - `\d*`: Any number of digits before the decimal point.
  - `\.?`: Optional decimal point.
  - `\d+`: At least one digit after the decimal point or any whole number.

It returns a list of all the numerals found in the sentence.

In [20]:
def extract_numerals(sentence):
    """Extract numerals and categorize them."""
    numerals = re.findall(r"[-]?\d*\.?\d+", sentence)
    return numerals

### `categorize_numeral` Function:

This function assigns a category to each numeral in the sentence based on the surrounding context.

- **Steps**:
  - The sentence is converted to lowercase for consistent comparison.
  - For each numeral, the position in the sentence is located.
  - A window of 10 characters around the numeral is captured to examine the local context.

- **Categorization Rules**:
  - If the local context contains '%' or words like "percent" or "per cent", the numeral is categorized as **percentage**.
  - If the context has symbols like '$', '€', '₹', or words like "dollar" or "rs", it is categorized as **monetary**.
  - If the numeral is exactly 4 digits (e.g., a year), it's classified as **temporal**.
  - Otherwise, it is classified as **other**. 

The function returns a list of categories for all numerals in the sentence.

In [21]:
def categorize_numeral(numerals, sentence):
    """Categorize each numeral in the sentence based on its local context."""
    sentence = sentence.lower()
   
    categories = []
    for numeral in numerals:
        numeral_pos = sentence.find(numeral)  # Find the position of the numeral
        window_size = 10  # Number of characters to look around the numeral
        # Get local context around the numeral
        start = max(0, numeral_pos - window_size)
        end = min(len(sentence), numeral_pos + len(numeral) + window_size)
        local_context = sentence[start:end]
        
        # Categorization rules
        if '%' in local_context or 'percent' in local_context or 'per cent' in local_context:
            categories.append('percentage')
        elif '$' in local_context or 'dollar' in local_context or '€' in local_context or 'rs' in local_context or '₹' in local_context:
            categories.append('monetary')
        elif re.match(r'^\d{4}$', numeral):  # Exactly 4 digits indicate temporal
            categories.append('temporal')
        else:
            categories.append('other')
    return categories

### `calculate_aspect_distance` Function:

This function calculates the distance between a **numeral** and its related **aspect** (main subject) in a sentence, using spaCy's dependency parsing.

- **Steps**:
  - The sentence is processed using spaCy to create a dependency tree.
  - The **aspect** (main subject) is identified, typically the first noun with a subject dependency (`nsubj` or `nsubjpass`).
  - If no subject is found, the first noun or proper noun in the sentence is used as a fallback aspect.
  - The function calculates the distance between the aspect and any numerals in the sentence.

- **Distance Calculation**:
  - For each numeral in the sentence, the absolute distance between the numeral and the aspect is measured in terms of token positions.
  - If the distance exceeds a predefined maximum (e.g., 10), it is capped at that value.
  - If no numeral is found, the default distance is 0.

The function returns the minimum distance between the aspect and the numerals.

In [22]:
def calculate_aspect_distance(sentence, numeral):
    """
    Calculate aspect distance from the numeral using spaCy's dependency tree.
    For simplicity, assuming 'aspect' is the main subject of the sentence.
    """
    doc = nlp(sentence)
    # Identify the main subject as the aspect
    aspect = None
    for token in doc:
        if token.dep_ in ('nsubj', 'nsubjpass') and token.pos_ in ('NOUN', 'PROPN'):
            aspect = token
            break
    if not aspect:
        # Fallback: choose the first noun as aspect
        for token in doc:
            if token.pos_ in ('NOUN', 'PROPN'):
                aspect = token
                break
    if not aspect:
        return 0  # Default distance if aspect is not found
    
    distances = []
    for token in doc:
        if numeral in token.text:
            distance = abs(aspect.i - token.i)
            distances.append(distance if distance < MAX_ASPECT_DISTANCE else MAX_ASPECT_DISTANCE - 1)
    if not distances:
        return 0
    return min(distances) 

### `get_embeddings` Function:

This function processes a sentence to extract numerals, categorize them, and compute aspect distances. It produces three types of embeddings needed for the sentiment analysis model:

1. **Character Tensor**:
   - Extracts numerals from the sentence and converts each digit into a unique index (e.g., '0' -> 1, '.', '-' also get unique indices).
   - Pads or truncates each numeral to a fixed length (e.g., 10 characters).

2. **Category Tensor**:
   - Uses the **`categorize_numeral`** function to classify each numeral (e.g., percentage, monetary, etc.).
   - Converts categories into numeric indices using a **label encoder**.

3. **Distance Tensor**:
   - Computes the distance between each numeral and its related aspect using **`calculate_aspect_distance`**.
   - Clips the distance to a maximum value (e.g., 9).

### Output:
- **`chars_tensor`**: Encodes each numeral as a fixed-length sequence of characters.
- **`categories_tensor`**: Encodes the category of each numeral.
- **`distances_tensor`**: Encodes the distance of each numeral from the aspect in the sentence.

This helps numerically encode the relationship between the numerals and the aspect for further analysis.

In [23]:
import torch

def get_embeddings(sentence, label_encoder, fixed_length=10, max_numerals=5):
    """
    Function to extract numerals, categorize them, and compute aspect distances for a given sentence.
    It returns three embeddings: 
    - Character tensor
    - Category tensor
    - Distance tensor
    """
    # Extract numerals and categories
    numerals = extract_numerals(sentence)
    categories = categorize_numeral(numerals, sentence)
    distances = [calculate_aspect_distance(sentence, num) for num in numerals]

    # Limit the number of numerals per sentence (optional)
    num_numerals = len(numerals)
    if num_numerals > max_numerals:
        numerals = numerals[:max_numerals]
        categories = categories[:max_numerals]
        distances = distances[:max_numerals]
        num_numerals = max_numerals

    # Padding numerals to a fixed length and converting to indices
    padded_chars = []
    for num in numerals:
        chars = list(num)
        char_indices = []
        for c in chars:
            if c == '.':
                char_indices.append(11)
            elif c == '-':
                char_indices.append(12)
            else:
                char_indices.append(int(c) + 1)  # '0' ->1, '1'->2, ..., '9'->10

        # Left-pad with 0 to the fixed length
        if len(char_indices) > fixed_length:
            char_indices = char_indices[-fixed_length:]
        else:
            char_indices = [0] * (fixed_length - len(char_indices)) + char_indices
        padded_chars.append(char_indices)

    # If fewer numerals than max_numerals, pad with zeros and default categories/distances
    while len(padded_chars) < max_numerals:
        padded_chars.append([0] * fixed_length)
        categories.append('other')  # Default category
        distances.append(0)  # Default distance

    # Convert categories to indices
    category_indices = label_encoder.transform(categories)

    # Clip distances to a maximum value
    clipped_distances = [min(d, MAX_ASPECT_DISTANCE - 1) for d in distances]

    # Convert to tensors
    chars_tensor = torch.tensor(padded_chars, dtype=torch.long)  # (max_numerals, fixed_length)
    categories_tensor = torch.tensor(category_indices, dtype=torch.long)  # (max_numerals,)
    distances_tensor = torch.tensor(clipped_distances, dtype=torch.long)  # (max_numerals,)

    return chars_tensor, categories_tensor, distances_tensor


In [24]:
sent=ndata['news'][1]
print(sent)

Apollo Hospitals Enterprise today reported a 7.4 per cent decline in standalone net profit at ₹ 67.44 crore for the third quarter ended December 31, 2017 on account of higher expenses. The company had


In [25]:
numerals=extract_numerals(sent)
print(numerals)

['7.4', '67.44', '31', '2017']


In [26]:
categories = categorize_numeral(numerals, sent)
print(categories)

['percentage', 'monetary', 'other', 'temporal']


In [29]:
distances = [calculate_aspect_distance(sent, num) for num in numerals]
print (distances)

[4, 9, 9, 9]


In [30]:
label_encoder=LabelEncoder()
label_encoder.fit(['percentage', 'monetary', 'temporal', 'other'])
print(get_embeddings(sent,label_encoder))

(tensor([[ 0,  0,  0,  0,  0,  0,  0,  8, 11,  5],
        [ 0,  0,  0,  0,  0,  7,  8, 11,  5,  5],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  4,  2],
        [ 0,  0,  0,  0,  0,  0,  3,  1,  2,  8],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0]]), tensor([2, 0, 1, 3, 1]), tensor([4, 9, 9, 9, 0]))


#### This embedding's where used as input for the CNN model which generates a unified emebedding's that incorporate's the numericao information of the sentence  and output embeddings from the CNN model is used for calculating aspect based sentiment analysis ,we are not finished with the aspect based sentiment analysis model yet we will be finishing it very soon.

#### Below is the architecture we are going to use for the CNN model ,we are yet to finalise it 

In [ ]:
# 3. Embedding Functions
class NumeralEncoder(nn.Module):
    """CNN-based Numeral Encoder for Financial News."""
    def __init__(self, char_embed_size=10, cat_embed_size=5, dis_embed_size=5, num_kernels=16, kernel_size=3):
        super(NumeralEncoder, self).__init__()
        # Character embedding layer (0-9, '.', '-'), total 12 possible characters
        self.char_embed = nn.Embedding(12, char_embed_size, padding_idx=0)
        
        # Category embedding (percentage, monetary, temporal, other)
        self.cat_label_encoder = LabelEncoder()
        self.cat_label_encoder.fit(['percentage', 'monetary', 'temporal', 'other'])
        self.cat_embed = nn.Embedding(len(self.cat_label_encoder.classes_), cat_embed_size)
        
        # Aspect distance embedding
        self.dis_embed = nn.Embedding(MAX_ASPECT_DISTANCE, dis_embed_size)
        
        # Convolutional layer
        self.conv = nn.Conv2d(1, num_kernels, kernel_size=(kernel_size, char_embed_size + cat_embed_size + dis_embed_size))
        self.pool = nn.MaxPool2d((2, 1))  # Adjust pooling as needed
        self.flatten = nn.Flatten()
        
    def forward(self, chars, categories, distances):
        """
        chars: Tensor of shape (batch_size, num_numerals, o)
        categories: Tensor of shape (batch_size, num_numerals)
        distances: Tensor of shape (batch_size, num_numerals)
        """
        batch_size, num_numerals, o = chars.size()
        # Reshape for embedding lookup
        chars = chars.view(-1, o)  # (batch_size * num_numerals, o)
        categories = categories.view(-1)
        distances = distances.view(-1)
        
        # Embeddings
        char_embeds = self.char_embed(chars)  # (batch_size * num_numerals, o, char_embed_size)
        cat_embeds = self.cat_embed(categories)  # (batch_size * num_numerals, cat_embed_size)
        dis_embeds = self.dis_embed(distances)  # (batch_size * num_numerals, dis_embed_size)
        
        # Expand cat_embeds and dis_embeds to match character embeddings
        cat_embeds = cat_embeds.unsqueeze(1).repeat(1, o, 1)  # (batch_size * num_numerals, o, cat_embed_size)
        dis_embeds = dis_embeds.unsqueeze(1).repeat(1, o, 1)  # (batch_size * num_numerals, o, dis_embed_size)
        
        # Concatenate embeddings
        combined_embeds = torch.cat([char_embeds, cat_embeds, dis_embeds], dim=2)  # (batch_size * num_numerals, o, total_embed_size)
        
        # Add channel dimension for CNN
        combined_embeds = combined_embeds.unsqueeze(1)  # (batch_size * num_numerals, 1, o, total_embed_size)
        
        # Convolution
        conv_out = self.conv(combined_embeds)  # (batch_size * num_numerals, num_kernels, o - kernel_size +1, 1)
        conv_out = torch.relu(conv_out)
        
        # Pooling
        pooled_out = self.pool(conv_out).squeeze(3)  # (batch_size * num_numerals, num_kernels, pooled_height)
        
        # Flatten
        flattened = self.flatten(pooled_out)  # (batch_size * num_numerals, num_kernels * pooled_height)
        
        # Reshape back to (batch_size, num_numerals, -1)
        final_embedding = flattened.view(batch_size, num_numerals, -1)
        return final_embedding  # (batch_size, num_numerals, num_kernels * pooled_height)
